In [3]:
import os, sys, datetime, logging
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px
import datetime
import calendar
from datetime import timedelta
import joblib
from tqdm import tqdm
from decimal import Decimal, getcontext

sys.path.append(os.getcwd().split('paper_std')[0])
if '../' not in sys.path:
    sys.path.append('../')

from util.s3_method import *
from util.time_method import *
from util.plot_method import easy_plot,timeline_sample_kv, get_plot_diff_data
from util.date import tz, pre_quarter_friday, next_quarter_friday
from util.hedge_log import initlog
from util.recover_depth import recoveryDepth
from util.statistic_method_v2 import describe_series
from util.load_s3_data import *
from dateutil.relativedelta import relativedelta
from util.Future_Load import *

根据Trade和ticker数据（默认使用2024-1-31-Binance-eth_udst）讨论每笔Trade数据对同时刻、最近的上一时刻ticker数据的偏离程度:
* tick_len 可设置的每档变动单位,为数字例如：0.01, 2, etc.（默认使用第一笔trade数据的变动幅度: keep_first_nonzero计算）
* total_num为对偏离程度统计的档数，默认有5档，偏离程度超过5*tick_len均设为第5档


In [14]:
def get_data(start_date, end_date, exchange, symbol):
    trade = FutureData.thread_get(FutureData.get_cex_trade, start_date, end_date, symbol, exchange, plot_interval_us = None)
    ticker = FutureData.thread_get(FutureData.get_cex_ticker, start_date, end_date, symbol, exchange, plot_interval_us = None)
    trade = pd.DataFrame(trade)
    ticker = pd.DataFrame(ticker).rename(columns={'tp':'time','ap':'ask1','aa':'askqty1','bp':'bid1','ba':'bidqty1'})
    trade = trade[['T','p','q','m']].rename(columns = {'p':'price','q':'qty','T':'time','m':'is_buyer_maker'})
    return ticker, trade

def keep_first_nonzero(value_list):
    '''辅助函数，用来找到第一位非零的位数作为tick_len'''
    not_zero = value_list != 0
    if sum(not_zero) == 0:
        print('输入交易价差全部为0，tick_len默认设置为0.01')
        return 0.01
    else:
        value = value_list[not_zero][0]
        if value < 1 :
            value_str = '%f'%(value)
            nonzero_index = next((i for i, c in enumerate(value_str) if c != '0' and c != '.'), None)
            print(f'tick_len设置为{float(value_str[:nonzero_index + 1])}')
            return float(value_str[:nonzero_index + 1])  # 只保留第一个非零数字和后面的数字
        else:
            print(f'tick_len设置为{value//1}')
            return value//1
    
def calculate_dif(price1, price2, tick_len):
    precision = min(Decimal(str(price1)).as_tuple().exponent, Decimal(str(price2)).as_tuple().exponent)
    return max((round(price1, abs(precision)) - round(price2, abs(precision)))//tick_len,0)

def get_dif_num(ticker, trade, tick_len = None, total_num=5):
    bid_num, ask_num = {}, {}
    for i in range(total_num):
        bid_num[f'bid{i+1}'] = 0
        ask_num[f'ask{i+1}'] = 0
    if tick_len == None:
        tick_len = keep_first_nonzero(np.abs(np.diff(trade.loc[0:10, 'price'])))
    for i in tqdm(range(len(trade))):
        timestamp, price, qty, is_buyer = trade.iloc[i,:]
        if timestamp>=ticker.time.values[0]:
            ticker_sub = ticker[ticker.time<=timestamp].iloc[-1,:]
            if is_buyer == 'BUY':
                num = calculate_dif(price, ticker_sub['ask1'], tick_len)
                if num < total_num:
                    bid_num[f'bid{int(num+1)}'] += 1
                else:
                    bid_num[f'bid{total_num}'] += 1
            else:
                num = calculate_dif(ticker_sub['bid1'], price, tick_len)
                if num < total_num:
                    ask_num[f'ask{int(num+1)}'] += 1
                else:
                    ask_num[f'ask{total_num}'] += 1
    return bid_num, ask_num

def solve_percentile(dic_num, direction):
    total = 0
    for key in dic_num.keys():
        total+=dic_num[key]
    print(f'总共{direction}成交盘口{total}笔：')
    for key in dic_num.keys():
        print(f'{key}档位占比{dic_num[key]/total: .2f}')


In [12]:
start_date = datetime.datetime(2024,3,14,10,tzinfo=TZ_8)
end_date = datetime.datetime(2024,3,14,11, tzinfo=TZ_8)
exchange, symbol ="binance", "bnb_usdt"
ticker,trade = get_data(start_date, end_date, exchange, symbol)
tick_len = None
total_num = 20

In [15]:
bid_num, ask_num = get_dif_num(ticker, trade, tick_len, total_num)
solve_percentile(bid_num, 'BUY')
solve_percentile(ask_num, 'SELL')

tick_len设置为0.1


100%|██████████| 39615/39615 [01:09<00:00, 567.17it/s]

总共BUY成交盘口21251笔：
bid1档位占比 0.93
bid2档位占比 0.04
bid3档位占比 0.02
bid4档位占比 0.01
bid5档位占比 0.00
bid6档位占比 0.00
bid7档位占比 0.00
bid8档位占比 0.00
bid9档位占比 0.00
bid10档位占比 0.00
bid11档位占比 0.00
bid12档位占比 0.00
bid13档位占比 0.00
bid14档位占比 0.00
bid15档位占比 0.00
bid16档位占比 0.00
bid17档位占比 0.00
bid18档位占比 0.00
bid19档位占比 0.00
bid20档位占比 0.00
总共SELL成交盘口18364笔：
ask1档位占比 0.88
ask2档位占比 0.07
ask3档位占比 0.03
ask4档位占比 0.01
ask5档位占比 0.01
ask6档位占比 0.00
ask7档位占比 0.00
ask8档位占比 0.00
ask9档位占比 0.00
ask10档位占比 0.00
ask11档位占比 0.00
ask12档位占比 0.00
ask13档位占比 0.00
ask14档位占比 0.00
ask15档位占比 0.00
ask16档位占比 0.00
ask17档位占比 0.00
ask18档位占比 0.00
ask19档位占比 0.00
ask20档位占比 0.00


In [ ]:
start_date = datetime.datetime(2024,3,14,10,tzinfo=TZ_8)
end_date = datetime.datetime(2024,3,14,11, tzinfo=TZ_8)
exchange, symbol ="binance", "eth_usdt"
tick_len = None
total_num = 20
for symbol in ['bnb_usdt_uswap', 'btc_usdt', 'eth_usdt']: #, 'bnb_usdt_uswap', 'eth_usdt', 'eth_usdt_uswap','btc_usdt', 'btc_usdt_uswap',]:
    ticker,trade = get_data(start_date, end_date, exchange, symbol)
    tick_len = keep_first_nonzero(np.abs(np.diff(trade.loc[0:10, 'price'])))    
    print(f"{exchange}:{symbol} {trade.iloc[0].price} {tick_len/trade.iloc[0].price}")
    bid_num, ask_num = get_dif_num(ticker, trade, tick_len, total_num)
    solve_percentile(bid_num, 'BUY')
    solve_percentile(ask_num, 'SELL')

tick_len设置为0.01
binance:bnb_usdt_uswap 618.3 1.617337861879347e-05


100%|██████████| 142656/142656 [09:45<00:00, 243.59it/s]


总共BUY成交盘口71023笔：
bid1档位占比 0.71
bid2档位占比 0.04
bid3档位占比 0.03
bid4档位占比 0.03
bid5档位占比 0.02
bid6档位占比 0.02
bid7档位占比 0.01
bid8档位占比 0.01
bid9档位占比 0.01
bid10档位占比 0.01
bid11档位占比 0.01
bid12档位占比 0.01
bid13档位占比 0.01
bid14档位占比 0.01
bid15档位占比 0.00
bid16档位占比 0.00
bid17档位占比 0.00
bid18档位占比 0.00
bid19档位占比 0.00
bid20档位占比 0.05
总共SELL成交盘口71632笔：
ask1档位占比 0.69
ask2档位占比 0.04
ask3档位占比 0.03
ask4档位占比 0.02
ask5档位占比 0.02
ask6档位占比 0.02
ask7档位占比 0.02
ask8档位占比 0.01
ask9档位占比 0.01
ask10档位占比 0.01
ask11档位占比 0.01
ask12档位占比 0.01
ask13档位占比 0.01
ask14档位占比 0.00
ask15档位占比 0.00
ask16档位占比 0.00
ask17档位占比 0.00
ask18档位占比 0.00
ask19档位占比 0.00
ask20档位占比 0.09
tick_len设置为0.01
binance:btc_usdt 72928.32 1.3712094286554247e-07


100%|██████████| 54583/54583 [01:54<00:00, 478.21it/s]


总共BUY成交盘口27683笔：
bid1档位占比 0.67
bid2档位占比 0.00
bid3档位占比 0.01
bid4档位占比 0.00
bid5档位占比 0.00
bid6档位占比 0.01
bid7档位占比 0.00
bid8档位占比 0.00
bid9档位占比 0.00
bid10档位占比 0.00
bid11档位占比 0.00
bid12档位占比 0.00
bid13档位占比 0.00
bid14档位占比 0.00
bid15档位占比 0.00
bid16档位占比 0.00
bid17档位占比 0.00
bid18档位占比 0.00
bid19档位占比 0.00
bid20档位占比 0.30
总共SELL成交盘口26896笔：
ask1档位占比 0.81
ask2档位占比 0.00
ask3档位占比 0.00
ask4档位占比 0.00
ask5档位占比 0.00
ask6档位占比 0.00
ask7档位占比 0.00
ask8档位占比 0.00
ask9档位占比 0.00
ask10档位占比 0.00
ask11档位占比 0.00
ask12档位占比 0.00
ask13档位占比 0.00
ask14档位占比 0.00
ask15档位占比 0.00
ask16档位占比 0.00
ask17档位占比 0.00
ask18档位占比 0.00
ask19档位占比 0.00
ask20档位占比 0.18
tick_len设置为0.01
binance:eth_usdt 3983.89 2.510109465873807e-06


 99%|█████████▉| 36619/37066 [01:12<00:01, 303.63it/s]

In [ ]:
start_date = datetime.datetime(2024,1,31,0,tzinfo=TZ_8)
end_date = datetime.datetime(2024,1,31,1, tzinfo=TZ_8)
exchange, symbol ="binance", "eth_usdt"
tick_len = None
total_num = 20
for symbol in [ 'eth_usdt', 'eth_usdt_uswap']:
    ticker,trade = get_data(start_date, end_date, exchange, symbol)
    tick_len = keep_first_nonzero(np.abs(np.diff(trade.loc[0:10, 'price'])))    
    print(f"{exchange}:{symbol} {trade.iloc[0].price} {tick_len/trade.iloc[0].price}")
    bid_num, ask_num = get_dif_num(ticker, trade, tick_len, total_num)
    solve_percentile(bid_num, 'BUY')
    solve_percentile(ask_num, 'SELL')

tick_len设置为0.01
binance:eth_usdt 2340.91 4.27184300122602e-06


100%|██████████| 77114/77114 [05:13<00:00, 245.59it/s]


总共BUY成交盘口41693笔：
bid1档位占比 0.62
bid2档位占比 0.06
bid3档位占比 0.02
bid4档位占比 0.05
bid5档位占比 0.01
bid6档位占比 0.04
bid7档位占比 0.01
bid8档位占比 0.02
bid9档位占比 0.01
bid10档位占比 0.02
bid11档位占比 0.00
bid12档位占比 0.01
bid13档位占比 0.00
bid14档位占比 0.01
bid15档位占比 0.00
bid16档位占比 0.01
bid17档位占比 0.00
bid18档位占比 0.01
bid19档位占比 0.00
bid20档位占比 0.08
总共SELL成交盘口35418笔：
ask1档位占比 0.67
ask2档位占比 0.05
ask3档位占比 0.02
ask4档位占比 0.04
ask5档位占比 0.01
ask6档位占比 0.03
ask7档位占比 0.01
ask8档位占比 0.02
ask9档位占比 0.01
ask10档位占比 0.02
ask11档位占比 0.00
ask12档位占比 0.01
ask13档位占比 0.00
ask14档位占比 0.01
ask15档位占比 0.00
ask16档位占比 0.01
ask17档位占比 0.00
ask18档位占比 0.01
ask19档位占比 0.00
ask20档位占比 0.07
输入交易价差全部为0，tick_len默认设置为0.01
binance:eth_usdt_uswap 2340.43 4.2727191157180525e-06


 73%|███████▎  | 236823/325304 [1:01:58<22:50:44,  1.08it/s]